In [8]:
from PIL import Image

def count_unique_colors(image_path):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to RGB mode if it's not already in RGB
    image = image.convert("RGB")

    # Get the color histogram of the image
    histogram = image.histogram()

    # Count the number of non-zero histogram values
    unique_colors = sum(1 for value in histogram if value != 0)

    return unique_colors

# Example usage:
image_path = "background_1.jpg"  # Replace this with the path to your JPG image
num_colors = count_unique_colors(image_path)
print("Number of unique colors:", num_colors)


Number of unique colors: 768


In [57]:
from PIL import Image

def jpg_to_coe(image_path, output_path):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to RGB mode if it's not already in RGB
    image = image.convert("RGB")

    # Get image dimensions
    width, height = image.size

    # Open the output COE file for writing
    with open(output_path, 'w') as f:
        # Write COE file header
        f.write("memory_initialization_radix=16;\n")
        f.write("memory_initialization_vector=\n")

        # Iterate over each pixel in the image
        for y in range(height):
            for x in range(width):
                # Get the RGB values of the pixel
                r, g, b = image.getpixel((x, y))

                # Convert RGB values to hexadecimal and write to COE file
                hex_value = f"{r:02X}{g:02X}{b:02X}"  # Convert to hex format
                f.write(hex_value)

                # Add comma after each pixel value except the last one
                if y != height - 1 or x != width - 1:
                    f.write(",")

            # Add newline after each row of pixels
            if y != height - 1:
                f.write("\n")

        # Write COE file footer
        f.write(";\n")

# Example usage:
input_image_path = "barrier_bear.jpg"  # Replace this with the path to your JPG image
output_coe_path = "barrier_bear.coe"  # Specify the output path for the COE file
jpg_to_coe(input_image_path, output_coe_path)


In [15]:
## New coe converter ##
from PIL import Image

def jpg_to_coe(jpg_file, coe_file):
    # Open the JPG image
    img = Image.open(jpg_file)
    
    # Get image dimensions
    width, height = img.size
    
    # Open the COE file for writing
    with open(coe_file, 'w') as f:
        # Write COE file header
        f.write("memory_initialization_radix=16;\n")
        f.write("memory_initialization_vector=\n")
        
        # Loop through each pixel in the image
        for y in range(height):
            for x in range(width):
                # Get RGB values of the pixel
                r, g, b = img.getpixel((x, y))
                
                # Convert RGB values to hexadecimal
                hex_value = '{:02X}{:02X}{:02X}'.format(r, g, b)
                
                # Write the hexadecimal value to the COE file
                f.write(hex_value)
                
                # Add a comma if it's not the last pixel
                if x != width - 1 or y != height - 1:
                    f.write(" ")
                else:
                    f.write(";")
            
            # Add a new line after each row of pixels
            f.write("\n")
    
    print("Conversion completed successfully.")

# Example usage:
# jpg_to_coe("Sprites/kun_1.jpg", "Sprites/kun_1.coe")
jpg_to_coe("dragon.jpg", "dragon.coe")

Conversion completed successfully.


In [17]:
############# 4 bit  each #######
from PIL import Image

def scale_to_4bit(value):
    # Scale the value from 0-255 to 0-15
    return value // 16

def jpg_to_coe(jpg_file, coe_file):
    # Open the JPG image
    img = Image.open(jpg_file)
    
    # Get image dimensions
    width, height = img.size
    
    # Open the COE file for writing
    with open(coe_file, 'w') as f:
        # Write COE file header
        f.write("memory_initialization_radix=16;\n")
        f.write("memory_initialization_vector=\n")
        
        # Loop through each pixel in the image
        for y in range(height):
            for x in range(width):
                # Get RGB values of the pixel
                r, g, b = img.getpixel((x, y))
                
                # Convert RGB values to 4-bit (hexadecimal)
                r4 = scale_to_4bit(r)
                g4 = scale_to_4bit(g)
                b4 = scale_to_4bit(b)
                
                # Combine 4-bit values into a single hexadecimal value
                hex_value = '{:X}'.format((r4 << 8) | (g4 << 4) | b4)
                
                # Write the hexadecimal value to the COE file
                f.write(hex_value)
                
                # Add a comma if it's not the last pixel
                if x != width - 1 or y != height - 1:
                    f.write(", ")
                else:
                    f.write(";")
            
            # Add a new line after each row of pixels
            f.write("\n")
    
    print("Conversion completed successfully.")

# Example usage:
jpg_to_coe("dragon.jpg", "dragon.coe")


Conversion completed successfully.


In [44]:
def parse_coe_file(file_path):
    colors = set()
    with open(file_path, 'r') as file:
        found_data = False
        for line in file:
            if line.strip().startswith('memory_initialization_radix'):
                continue
            elif line.strip().startswith('memory_initialization_vector'):
                found_data = True
                continue
            elif found_data:
                # Extract RGB values from each line
                rgb_values = line.strip().split(',')
                for rgb_value in rgb_values:
                    if rgb_value:
                        r = int(rgb_value[0:2], 16)
                        g = int(rgb_value[2:4], 16)
                        b = int(rgb_value[4:6], 16)
                        if (r,g,b) not in colors:
                            colors.add((r, g, b))
#                             print((r,g,b))
    return colors

def create_lookup_palette(colors):
    palette = {}
    for index, color in enumerate(colors):
#         print(color)
        palette[color] = index
    return palette

def replace_with_index_and_write(file_path, palette, output_file_path):
    with open(file_path, 'r') as file:
        with open(output_file_path, 'w') as output_file:
            for line_num, line in enumerate(file):
                if line_num < 2 or line.startswith(';'):
                    output_file.write(line)
                else:
                    # Extract RGB values and replace with index
                    rgb_values = [rgb_value.strip() for rgb_value in line.split(',')]
                    if rgb_values:  # Skip empty lines
                        replaced_line = ' '.join([f'{palette[int("".join(filter(str.isalnum, rgb_value)), 16)]:02X}' for rgb_value in rgb_values])
                        output_file.write(replaced_line + '\n')






# Parse COE file and find unique colors
coe_file_path = 'Background_2.coe'  # Replace with the path to your COE file
unique_colors = parse_coe_file(coe_file_path)
# print(len(unique_colors))
# print(unique_colors)
# print(unique_colors)
# Create lookup palette
lookup_palette = create_lookup_palette(unique_colors)
# print(lookup_palette)
# Specify the output COE file path
output_coe_file_path = 'output_coe_file.coe'  # Replace with the desired output file path

# Replace RGB values with index in COE file and write to a new file
replace_with_index_and_write(coe_file_path, lookup_palette, output_coe_file_path)


21866


KeyError: 16777215

In [37]:
rgb_value= "7FBBE0"
r = int(rgb_value[0:2], 16)
g = int(rgb_value[2:4], 16)
b = int(rgb_value[4:6], 16)
print((r, g, b))

(127, 187, 224)
